In [45]:
import pandas as pd

df = pd.read_json("msc_self_instruct.jsonl", lines=True)

In [158]:
answer_df = pd.DataFrame()
answer_df['dialog'] = None
answer_df['question'] = None
answer_df['gold_answer'] = None
answer_df['long_mem_result'] = None
answer_df['long_mem_answer'] = None
answer_df['long_mem_f1'] = None
answer_df['long_mem_rc'] = None
answer_df['long_mem_pre'] = None
answer_df['long_mem_recall_result'] = None
answer_df['long_mem_recall_answer'] = None
answer_df['long_mem_recall_f1'] = None
answer_df['long_mem_recall_rc'] = None
answer_df['long_mem_recall_pre'] = None

In [159]:
from datetime import datetime, timedelta

for i in range(500):
    # Add dialog
    current_time = datetime.strptime("2024/11/1 12:00", "%Y/%m/%d %H:%M")
    dialog_data = []
    for dialog_session in df['previous_dialogs'][i]:
        session = []
        for count in range(int(len(dialog_session['dialog'])/2)):
            chat_log = {"text":f"Allen:{dialog_session['dialog'][2*count]['text']}, Jack:{dialog_session['dialog'][2*count+1]['text']}", "time":current_time.strftime("%Y-%m-%dT%H:%M:%SZ")}
            session.append(chat_log)
            current_time = current_time + timedelta(minutes=1)
        dialog_data.append(session)
    answer_df.loc[i, 'dialog'] = str(dialog_data)
    
    # Add question & answer
    answer_df.loc[i, 'question'] = f"Jack:{df['self_instruct'][i]['B']}"
    answer_df.loc[i, 'gold_answer'] = f"{df['self_instruct'][i]['A']}"

In [160]:
answer_df.head()

,dialog,question,gold_answer,long_mem_result,long_mem_answer,long_mem_f1,long_mem_rc,long_mem_pre,long_mem_recall_result,long_mem_recall_answer,long_mem_recall_f1,long_mem_recall_rc,long_mem_recall_pre
0,"[[{'text': ""Allen:Hi! How are you doing tonigh...","Jack:Hey, remember that time we talked about m...",Taylor Swift!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[[{'text': 'Allen:Hello, how are you doing?, J...","Jack:Hey, remember that time we talked about o...",I eat a fresh and raw diet to save on groceries.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[[{'text': 'Allen:Hello what are doing today?,...","Jack:Hey, remember that time we talked about o...",I used to work in the human services field.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[[{'text': ""Allen:How are you? I'm tired of my...","Jack:Hey, remember that time we talked about o...",Burger King!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[[{'text': 'Allen:Hi, how are you doing today?...","Jack:Hey, remember that time we talked about o...",Three miles!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
answer_df.to_json("MSC_datasets.json", orient="records", lines=True)

In [41]:
import pandas as pd

answer_df = pd.read_json("MSC_datasets.json", lines=True)

Find result

In [43]:
import pandas as pd

answer_df = pd.read_json("MSC_eval.json", lines=True)

In [ ]:
from long_memory.component import WeaviateLongMemory
from short_memory.component import WeaviateShortMemory
long_mem = WeaviateLongMemory()
short_mem = WeaviateShortMemory()

In [4]:
other_instruct="""Your role is Allen, Jack will ask some question about you, you need to find relative memory in Allen's memory.
here's some guide for you. similar_snippets is original memory, related_summaries and closest_summary are compressed memory for suggestion, 
If you see some relative content from related_summaries, you can use jump to related_summaries and get original memory,
when you use retry, you can use specific things to search not just a question.
when you need to write evidence, try put original dialog into evidence field, not compressed memory"""

In [36]:
error_list = []
for i in range(100):
    print(f'==={i}===')
    try:
        long_mem.del_memory()
        dialogs = []
        for dialog_session in eval(answer_df.loc[i, 'dialog']):
            dialogs.extend(dialog_session)
        long_mem.add_chat_logs(dialogs)
        question = f"{answer_df.loc[i, 'question']}"
        answer_df.loc[i, 'long_mem_result'] = str(long_mem.get_memory(question, recall=False))
        answer_df.loc[i, 'long_mem_recall_result'] = str(long_mem.get_memory(question, recall=True, other_instruct=other_instruct))
    except Exception as e:
        error_list.append(i)
        print(f'----error:{e}----')
    if (i+1)%20==0:
        answer_df.to_json("MSC_eval.json", orient="records", lines=True)
answer_df.to_json("MSC_eval.json", orient="records", lines=True)

===97===
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
Save chat logs to long memory done.


In [ ]:
s_answer_df = pd.read_json("s_MSC_datasets.json", lines=True)
error_list = []
for i in range(100):
    print(f'==={i}===')
    try:
        short_mem.del_memory()
        dialogs = []
        for dialog_session in eval(s_answer_df.loc[i, 'dialog']):
            dialogs.extend(dialog_session)
        short_mem.add_chatlogs(dialogs)
        question = f"{answer_df.loc[i, 'question']}"
        s_answer_df.loc[i, 'short_mem_result'] = str(short_mem.get_memory(question))
    except Exception as e:
        error_list.append(i)
        print(f'----error:{e}----')
    if (i+1)%20==0:
        s_answer_df.to_json("s_MSC_eval.json", orient="records", lines=True)
s_answer_df.to_json("s_MSC_eval.json", orient="records", lines=True)

===0===
Detect empty short memory, create memory space...


/tmp/ipykernel_1432440/3636161595.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '{'retrieve_memory': [{'text': 'user:Cars are my thing. Vintage cars. I love working on them. Wrestling? Do you enjoy it?, assistant:Cool. Not really into cars. My day job is wrestling.', 'time': '11/01 12:03'}, {'text': 'user:I agree. Working out is a great way to burn off steam. Do you like country music?, assistant:Understood. When I need to be alone, I work out a few times each week.', 'time': '11/01 12:05'}, {'text': "user:Lol. She's more pop now. Her old stuff was country. I like classic country., assistant:A little bit. I can get into taylor swift.", 'time': '11/01 12:06'}, {'text': 'user:Well I do like to listen to classic country, and I do have my two dogs, "Baron Zemo" and "Spike."   What about you, how do you like wrestling as your day job?, assistant:This sounds fun. Do you have any other hobby?', 'time': '1

In [59]:
s_answer_df['short_mem_result'][0]

'{\'retrieve_memory\': [{\'text\': \'user:Cars are my thing. Vintage cars. I love working on them. Wrestling? Do you enjoy it?, assistant:Cool. Not really into cars. My day job is wrestling.\', \'time\': \'11/01 12:03\'}, {\'text\': \'user:I agree. Working out is a great way to burn off steam. Do you like country music?, assistant:Understood. When I need to be alone, I work out a few times each week.\', \'time\': \'11/01 12:05\'}, {\'text\': "user:Lol. She\'s more pop now. Her old stuff was country. I like classic country., assistant:A little bit. I can get into taylor swift.", \'time\': \'11/01 12:06\'}, {\'text\': \'user:Well I do like to listen to classic country, and I do have my two dogs, "Baron Zemo" and "Spike."   What about you, how do you like wrestling as your day job?, assistant:This sounds fun. Do you have any other hobby?\', \'time\': \'11/01 12:11\'}, {\'text\': "user:Yeah between the both of us, could probably get a country song written about our distresses...perhaps Tay

In [55]:
error_list

[]

Generate answer

In [9]:
answer_prompt = """Base on the following document and answer the question.
You are Allen, try use origin text in the evidence field to repeat a brief answer.

Question:{question}

Document:{docs}
"""

In [10]:
from dotenv import load_dotenv
from openai import OpenAI
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def llm_create(prompt):
        messages = [{"role": "user", "content": prompt}]
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
        )
        return completion.choices[0].message.content

In [ ]:
for i in range(100):
    print(f'==={i}===')
    p = answer_prompt.format(question=answer_df.loc[i, 'question'], docs=answer_df.loc[i, 'long_mem_result'])
    long_mem_answer = llm_create(p)
    answer_df.loc[i, 'long_mem_answer'] = long_mem_answer
    
    p = answer_prompt.format(question=answer_df.loc[i, 'question'], docs=answer_df.loc[i, 'long_mem_recall_result'])
    long_mem_recall_answer = llm_create(p)
    answer_df.loc[i, 'long_mem_recall_answer'] = long_mem_recall_answer
    
    if (i+1)%20==0:
        answer_df.to_json("MSC_eval.json", orient="records", lines=True)
answer_df.to_json("MSC_eval.json", orient="records", lines=True)

In [66]:
index=6
eval(answer_df['long_mem_recall_result'][index])

{'search times': 1,
 'used queries': ['Jack:Hey, remember that time we talked about our favorite movies? What was yours?'],
 'searched memory': [{'text': 'Allen and Jack discuss their preferences in movie genres, primarily comedies and nature documentaries.',
   'time': '2024/11/01 12:15'}],
 'thought': "I found detailed memories of our conversation about movie preferences, highlighting that I enjoy nature documentaries, particularly the BBC series 'Blue Planet II', while Jack enjoys comedies like 'Clueless'.",
 'evidence': [{'text': "Allen: I haven't. I'm not really into comedies, because I don't really have a sophisticated sense of humor. What genre do you think your first book will be?, Jack: it would be the kind of comedy that clueless is. Do you not think of clueless as a comedy?"},
  {'text': "Allen: Yeah, I think so. But what I enjoy the most is nature documentaries. I like learning new things, and so I'll sometimes spend the whole day knitting and watching wildlife on tv!"},
  

In [12]:
answer_df.head()

,dialog,question,gold_answer,long_mem_result,long_mem_answer,long_mem_f1,long_mem_rc,long_mem_pre,long_mem_recall_result,long_mem_recall_answer,long_mem_recall_f1,long_mem_recall_rc,long_mem_recall_pre
0,"[[{'text': ""Allen:Hi! How are you doing tonigh...","Jack:Hey, remember that time we talked about m...",Taylor Swift!,{'closest_summary': {'text': 'Allen and Jack t...,The document does not mention any specific art...,NaN,NaN,NaN,"{'search times': 4, 'used queries': ['Jack:Hey...",I mentioned that I could get into Taylor Swift.,NaN,NaN,NaN
1,"[[{'text': 'Allen:Hello, how are you doing?, J...","Jack:Hey, remember that time we talked about o...",I eat a fresh and raw diet to save on groceries.,{'closest_summary': {'text': 'Allen shares his...,Allen saved money by primarily eating a fresh ...,NaN,NaN,NaN,"{'search times': 1, 'used queries': ['Jack:Hey...",I mentioned that I mostly eat a fresh and raw ...,NaN,NaN,NaN
2,"[[{'text': 'Allen:Hello what are doing today?,...","Jack:Hey, remember that time we talked about o...",I used to work in the human services field.,"{'closest_summary': {'text': ""Allen and Jack d...",I used to work in the human services field.,NaN,NaN,NaN,"{'search times': 1, 'used queries': ['Jack:Hey...",Allen used to work in the human services field.,NaN,NaN,NaN
3,"[[{'text': ""Allen:How are you? I'm tired of my...","Jack:Hey, remember that time we talked about o...",Burger King!,{'closest_summary': {'text': 'Conversation abo...,Allen: I have a part-time job at Burger King.,NaN,NaN,NaN,"{'search times': 1, 'used queries': [""Jack:Hey...",Sure! I work at Burger King.,NaN,NaN,NaN
4,"[[{'text': 'Allen:Hi, how are you doing today?...","Jack:Hey, remember that time we talked about o...",Three miles!,{'closest_summary': {'text': 'Allen and Jack d...,I mentioned that I like to walk three miles fo...,NaN,NaN,NaN,"{'search times': 1, 'used queries': ['Jack:Hey...",I mentioned that I like to walk for a small wo...,NaN,NaN,NaN


ROUGE-L Score

In [37]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

score_df = answer_df[:100]

def evaluate(col):
    def f1(row):
        scores = scorer.score(row['gold_answer'], row[f'{col}_result'])
        return scores['rougeL'].fmeasure
    def rc(row):
        scores = scorer.score(row['gold_answer'], row[f'{col}_result'])
        return scores['rougeL'].recall
    def pre(row):
        scores = scorer.score(row['gold_answer'], row[f'{col}_result'])
        return scores['rougeL'].precision
    score_df[f'{col}_f1'] = score_df.apply(f1, axis=1)
    score_df[f'{col}_rc'] = score_df.apply(rc, axis=1)
    score_df[f'{col}_pre'] = score_df.apply(pre, axis=1)
evaluate('long_mem')
evaluate('long_mem_recall')

/tmp/ipykernel_1432440/2335146260.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df[f'{col}_f1'] = score_df.apply(f1, axis=1)
/tmp/ipykernel_1432440/2335146260.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df[f'{col}_rc'] = score_df.apply(rc, axis=1)
/tmp/ipykernel_1432440/2335146260.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

In [38]:
number = 100
long_mem_means = score_df[['long_mem_f1', 'long_mem_rc', 'long_mem_pre']][:number].mean()
long_mem_recall_means = score_df[['long_mem_recall_f1', 'long_mem_recall_rc', 'long_mem_recall_pre']][:number].mean()
print(long_mem_means)
print(long_mem_recall_means)

long_mem_f1     0.018149
long_mem_rc     0.697708
long_mem_pre    0.009285
dtype: float64
long_mem_recall_f1     0.047865
long_mem_recall_rc     0.755867
long_mem_recall_pre    0.025495
dtype: float64


In [144]:
score_df.to_json("MSC_score_v1.json", orient="records", lines=True)

In [30]:
score_df

,dialog,question,gold_answer,long_mem_result,long_mem_answer,long_mem_f1,long_mem_rc,long_mem_pre,long_mem_recall_result,long_mem_recall_answer,long_mem_recall_f1,long_mem_recall_rc,long_mem_recall_pre
0,"[[{'text': ""Allen:Hi! How are you doing tonigh...","Jack:Hey, remember that time we talked about m...",Taylor Swift!,{'closest_summary': {'text': 'Allen and Jack t...,The document does not mention any specific art...,0.000000,0.000000,0.000000,"{'search times': 4, 'used queries': ['Jack:Hey...",I mentioned that I could get into Taylor Swift.,0.021858,1.000000,0.011050
1,"[[{'text': 'Allen:Hello, how are you doing?, J...","Jack:Hey, remember that time we talked about o...",I eat a fresh and raw diet to save on groceries.,{'closest_summary': {'text': 'Allen shares his...,Allen saved money by primarily eating a fresh ...,0.056818,0.909091,0.029326,"{'search times': 1, 'used queries': ['Jack:Hey...",I mentioned that I mostly eat a fresh and raw ...,0.155039,0.909091,0.084746
2,"[[{'text': 'Allen:Hello what are doing today?,...","Jack:Hey, remember that time we talked about o...",I used to work in the human services field.,"{'closest_summary': {'text': ""Allen and Jack d...",I used to work in the human services field.,0.053892,1.000000,0.027692,"{'search times': 1, 'used queries': ['Jack:Hey...",Allen used to work in the human services field.,0.156522,1.000000,0.084906
3,"[[{'text': ""Allen:How are you? I'm tired of my...","Jack:Hey, remember that time we talked about o...",Burger King!,{'closest_summary': {'text': 'Conversation abo...,Allen: I have a part-time job at Burger King.,0.022727,1.000000,0.011494,"{'search times': 1, 'used queries': [""Jack:Hey...",Sure! I work at Burger King.,0.040000,1.000000,0.020408
4,"[[{'text': 'Allen:Hi, how are you doing today?...","Jack:Hey, remember that time we talked about o...",Three miles!,{'closest_summary': {'text': 'Allen and Jack d...,I mentioned that I like to walk three miles fo...,0.013423,1.000000,0.006757,"{'search times': 1, 'used queries': ['Jack:Hey...",I mentioned that I like to walk for a small wo...,0.038095,1.000000,0.019417
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,[[{'text': 'Allen:Good morning. My name is sop...,"Jack:Hey, remember that time we talked about l...",The alphabet!,{'closest_summary': {'text': 'Allen wishes Jac...,Allen mentioned that he is learning the alphab...,0.020833,1.000000,0.010526,"{'search times': 2, 'used queries': ['Jack:Hey...",Allen is planning to learn the alphabet.,0.033613,1.000000,0.017094
96,"[[{'text': 'Allen:Hello, how are you tonight?,...","Jack:Hey, remember that time we talked about t...",Fish tacos with cilantro lime slaw.,"{'closest_summary': {'text': ""Allen and Jack d...",Allen's fiancé makes fish tacos.,0.054545,1.000000,0.028037,"{'search times': 1, 'used queries': ['Jack:Hey...",Allen's fiancé makes a killer fish taco.,0.121212,1.000000,0.064516
97,"[[{'text': 'Allen:Hello, I finally got a chanc...","Jack:Hey, remember that time we talked about s...",Get married and obey your wife.,{'closest_summary': {'text': 'Allen shares his...,"Jack's third piece of advice was to ""Get marri...",0.037152,1.000000,0.018927,"{'search times': 2, 'used queries': ['Jack:Hey...",The third piece of advice you gave me was rela...,0.013245,0.166667,0.006897
98,"[[{'text': ""Allen:Hey, how are you? I was just...","Jack:Hey, remember that time we talked about y...",I'm paid entirely in broken watches.,"{'closest_summary': {'text': ""Allen talks abou...","Allen: I do sell watches, but I spend all my p...",0.017241,0.428571,0.008798,"{'search times': 1, 'used queries': ['Jack:Hey...",Allen: I spend all my paychecks on more watche...,0.068966,0.428571,0.037500


In [39]:
index=91
eval(answer_df['long_mem_result'][index])

{'closest_summary': {'text': 'Jack considers getting a dog, discussing suitable breeds.',
  'time': '2024/11/01 12:08'},
 'similar_snippets': [{'text': "Allen:My dogs really uplift my mood whenever I am down. Just yesterday, I had a horrible day but just spending time with the dogs made everything good. Have you considered getting dogs to help you?, Jack:I would consider a dog but I don't know where to start.  What breeds should I consider?",
   'time': '2024/11/01 12:08'},
  {'text': "Allen:You definitely don't want a breed that could get aggressive or that requires a considerable amount of training.  I find that golden retrievers, labs, and boxers are excellent companions.  I guess the other thing to think about is the size of the dog...would you want a smaller or more medium/large breed?, Jack:I would want one to keep in the house so it doesn't need to shed much.  I like the idea of a smaller dog but somewhat manly?  Andy idea of dogs based on that?",
   'time': '2024/11/01 12:09'},

In [25]:
score_df[score_df['long_mem_recall_rc']<0.5].index

Index([11, 12, 13, 15, 19, 23, 24, 33, 34, 36, 38, 39, 42, 44, 47, 48, 50, 55,
       61, 63, 68, 71, 73, 75, 76, 82, 84, 88, 89, 90, 92, 97, 98],
      dtype='int64')